In [ ]:
'''
2018년 만
key : station_nm
 lng: 경도
 lat: 위도
 line_num: 노선이름
 data:
   - list of (월별)
      - month
      - nums (시간별)
         - time 
         - ride 승차
         - alight 하차
'''

In [13]:
'''
UnicodeDecodeError: 'cp949' codec can't decode byte 0xe2 in position 6987: illegal multibyte sequence
encoding 설정 필요함
'''
import json
with open('data/seoul_metro_time.json', encoding='UTF8') as f:
    time_data = json.load(f)
with open('data/seoul_metro_loc.json', encoding='UTF8') as f:
    loc_data = json.load(f)

In [31]:
"""key들을 정렬해줘야 한다."""
# key, value 바꿈
desp = {time_data['DESCRIPTION'][k] : k for k in time_data['DESCRIPTION']}
ride_keys = []
alight_keys = []
for i in range(24):
    time_str = '%s시-%s'%(str(i).zfill(2),  # 0을 붙인다 앞에
                          str(i+1).zfill(2)) 
    ride_keys.append(desp[time_str + '시 승차인원'].lower())
    alight_keys.append(desp[time_str + '시 하차인원'].lower())

In [86]:
"""loc_data에 있는 정보들"""
res = {}
for st in loc_data['DATA']:
    res[st['station_nm']] = {'station_nm': st['station_nm'],
                             'lng': st['ypoint_wgs'],
                             'lat': st['xpoint_wgs'],
                             'line_num': st['line_num'],
                             'data': []}

In [87]:
"""time_data에 있는 정보들"""
for t in time_data['DATA']:
    if "2018" not in t['use_mon']:
        continue
    if t['sub_sta_nm'] not in res:
        continue
    # 환승역의 경우 여러번 나오므로 값을 합쳐준다.
    flag = False # 환승역일 경우 True
    if len(res[t['sub_sta_nm']]['data']) > 0 and \
       res[t['sub_sta_nm']]['data'][-1]['month'] == t['use_mon']:
        elem = res[t['sub_sta_nm']]['data'][-1]
        flag = True
    else:
        elem = {'month': t['use_mon'],
                'nums': []}
    # 시간정보를 list에 더해준다.
    for i in range(24):
        time_str = '%s시-%s시'%(str(i).zfill(2), str(i+1).zfill(2)) 
        if len(elem['nums']) > i:
            elem['nums'][i] = (time_str,
                               elem['nums'][i][1] + t[ride_keys[i]],
                               elem['nums'][i][2] + t[alight_keys[i]])
        else:
            elem['nums'].append((time_str,
                                 t[ride_keys[i]],
                                 t[alight_keys[i]]))
    if flag:
        res[t['sub_sta_nm']]['data'][-1] = elem
    else:
        res[t['sub_sta_nm']]['data'].append(elem)

In [88]:
"""dict를 list로 변환한다."""
res_list = []
for station in res:
    res_list.append(res[station])

In [90]:
"""write 한다."""
with open('data/seoul_metro2018.json', 'w', encoding='UTF8') as f:
    json.dump(res_list, f)